<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Cleaning-the-dataset" data-toc-modified-id="Cleaning-the-dataset-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Cleaning the dataset</a></span></li></ul></li><li><span><a href="#making-some-predicitons" data-toc-modified-id="making-some-predicitons-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>making some predicitons</a></span></li><li><span><a href="#Vasco's-code" data-toc-modified-id="Vasco's-code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Vasco's code</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

In [7]:
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('./dengue_features_train.csv')
labels = pd.read_csv('./dengue_labels_train.csv')

train['total_cases'] = labels['total_cases']

## Cleaning the dataset 

In [4]:
train = train.fillna(method='backfill')

In [14]:
train.city.unique()
train_sj = train.loc[train.city == 'sj']

In [15]:
train_iq = train.loc[train.city == 'iq']

In [16]:
train.city.unique()

array(['sj', 'iq'], dtype=object)

In [17]:
lag_list_sj = [2, 3, 6, 9]
lag_list_iq = [1, 3, 5]

In [18]:
lags = pd.DataFrame({'lag_{}'.format(x): train_sj.total_cases.shift(x) for x in lag_list}).iloc[9:]

NameError: name 'lag_list' is not defined

In [19]:
train_sj.iloc[9:].shape

(927, 25)

In [20]:
baseline = train_sj.iloc[9:].join(lags)

NameError: name 'lags' is not defined

In [21]:
baseline.head()

NameError: name 'baseline' is not defined

In [22]:
baseline = baseline.drop(['city', 'year', 'week_start_date'], axis=1)
baseline['cos_weekofyear'] = np.cos(baseline.weekofyear*(2*np.pi/52))
baseline['sin_weekofyear'] = np.sin(baseline.weekofyear*(2*np.pi/52))

NameError: name 'baseline' is not defined

In [23]:
def clean(df, lag_list):
    c = lag_list[-1]
    lags = pd.DataFrame({'lag_{}'.format(x): df.total_cases.shift(x) for x in lag_list}).iloc[c:]
    baseline = df.iloc[9:].join(lags)
    return baseline

baseline = clean(train_iq, lag_list)

NameError: name 'lag_list' is not defined

In [24]:
baseline.head()

NameError: name 'baseline' is not defined

In [25]:
baseline.shape

NameError: name 'baseline' is not defined

In [26]:
def regressor(df, test_size):
    df = clean(df, lag_list)
    df = df.drop(['city', 'year', 'week_start_date'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['total_cases'], axis=1),
                                                    df.total_cases,
                                                    test_size=test_size,
                                                    random_state=42
                                                   )
    
    regressor = RandomForestRegressor(random_state=42)
    regressor.fit(X_train, y_train)
    y_pred = np.round(regressor.predict(X_test)) if test_size else None
    return y_pred, regressor, X_test, y_test

In [27]:
y_pred_sj, regressor_sj, X_test_sj, y_test_sj = regressor(train_sj, test_size=0)
y_pred_iq, regressor_iq, X_test_iq, y_test_iq = regressor(train_iq, test_size=0)

NameError: name 'lag_list' is not defined

In [28]:
print('SJ', mean_absolute_error(y_test_sj, y_pred_sj))
print('IQ', mean_absolute_error(y_test_iq, y_pred_iq))

NameError: name 'y_test_sj' is not defined

In [29]:
plt.figure(figsize=(20,15))

feat_importances = pd.Series(regressor.feature_importances_, index=X_train.columns)
feat_importances.nlargest(10).plot(kind='barh')

AttributeError: 'function' object has no attribute 'feature_importances_'

<Figure size 1440x1080 with 0 Axes>

# making some predicitons

In [6]:
test = pd.read_csv('./dengue_features_test.csv')

In [114]:
test_sj = test.loc[test.city == 'sj']
test_iq = test.loc[test.city == 'iq']

In [121]:
test_sj = test_sj.drop(['city', 'year', 'week_start_date'], axis=1)
test_sj = test_sj.fillna(method='backfill')

In [122]:
regressor_sj.predict(test_sj)

ValueError: Number of features of the model must match the input. Model n_features is 24 and input n_features is 21 

# Vasco's code

In [80]:
def shift_magic(df):
    cities = df[['city']]
    # res = DataFrame(columns = df.columns) 
    res = pd.DataFrame()
    for city in ['iq', 'sj']:
        df_city = df[df.city == city].copy().set_index('week_start_date')
        df_city = df_city.drop('city', axis=1)
        df_city = df_city.join(df_city - df_city.shift(52), rsuffix='_shifted')
        df_city.index = df[df.city == city].index
        #print(df_city.precipitation_amt_mm  )
        res = res.append(df_city, ignore_index=False)
    res = res.join(df[['city']])
    return  res

In [81]:
def pipeline_1(t):
    t = shift_magic(t)
    t = pd.get_dummies(t)
    t = t.drop(['year', 'year_shifted'], axis=1)
    t['cos_weekofyear'] = np.cos(t.weekofyear*(2*np.pi/52))
    t['sin_weekofyear'] = np.sin(t.weekofyear*(2*np.pi/52))
    t = t.drop(['weekofyear'], axis=1)
    return t


In [33]:
t = pipeline_1(train)
X_train, y_train = t.drop(['total_cases', 'total_cases_shifted'], axis=1), t.total_cases

In [50]:
train_test = train.append(test, ignore_index=True, sort=False)
train_test = train_test.drop('total_cases', axis=1)

In [82]:
X_test = pipeline_1(train_test.fillna(method='backfill'))

In [87]:
train_test.shape

(1872, 24)

In [84]:
X_test.shape

(1872, 45)

In [85]:
X_test.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_tdtr_k_shifted,station_avg_temp_c_shifted,station_diur_temp_rng_c_shifted,station_max_temp_c_shifted,station_min_temp_c_shifted,station_precip_mm_shifted,city_iq,city_sj,cos_weekofyear,sin_weekofyear
936,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,298.450000,295.184286,307.3,293.1,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-1.000000,1.224647e-16
937,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,298.428571,295.358571,306.6,291.1,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-0.992709,-1.205367e-01
938,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,297.392857,295.622857,304.5,292.6,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-0.970942,-2.393157e-01
939,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,296.228571,292.797143,303.6,288.6,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-0.935016,-3.546049e-01
940,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,297.635714,293.957143,307.0,291.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-0.885456,-4.647232e-01


In [37]:
r = RandomForestRegressor(random_state=42)

In [38]:
r.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [88]:
y_pred = np.round(r.predict(X_test.fillna(method='backfill')))

In [89]:
X_test['total_cases'] = y_pred

In [90]:
X_test[['city', 'year', 'weekofyear', 'total_cases']]

KeyError: "['city' 'year' 'weekofyear'] not in index"

In [105]:
X_test.iloc[-416:].sort_values(by='ndvi_ne')

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,station_diur_temp_rng_c_shifted,station_max_temp_c_shifted,station_min_temp_c_shifted,station_precip_mm_shifted,city_iq,city_sj,cos_weekofyear,sin_weekofyear,total_cases,cheat
1641,-0.463400,-0.033100,0.102786,0.061100,44.02,300.134286,300.307143,296.257143,302.0,298.4,...,1.285714,1.7,0.5,-16.8,0,1,0.748511,-6.631227e-01,35.0,"(-0.4634, -0.0331, 0.1027857, 0.0611)"
1696,-0.364000,0.016300,0.190243,0.159857,3.08,299.331429,299.271429,295.247143,301.4,297.5,...,0.485714,2.2,1.6,-34.8,0,1,0.935016,-3.546049e-01,57.0,"(-0.364, 0.0163, 0.1902429, 0.1598571)"
1697,-0.364000,-0.142900,0.067257,0.065857,34.66,299.150000,299.200000,294.357143,300.9,297.1,...,1.085714,2.8,0.0,-61.1,0,1,0.970942,-2.393157e-01,60.0,"(-0.364, -0.1429, 0.06725714, 0.06585714)"
929,-0.321400,-0.141200,0.110643,0.141014,0.00,297.595714,297.642857,292.197143,299.5,296.1,...,-2.228571,-2.8,0.5,4.1,0,1,0.239316,9.709418e-01,8.0,"(-0.3214, -0.1412, 0.11064289999999999, 0.1410..."
1699,-0.309800,-0.112050,0.143671,0.140814,31.83,298.295714,298.414286,294.692857,300.3,296.0,...,0.800000,1.7,-0.5,-25.4,0,1,0.992709,1.205367e-01,38.0,"(-0.3098, -0.11205, 0.1436714, 0.1408142999999..."
821,-0.308400,-0.136900,0.266100,0.210457,18.15,297.825714,297.992857,293.457143,300.1,296.5,...,0.400000,1.6,0.0,16.5,0,1,0.748511,6.631227e-01,9.0,"(-0.3084, -0.1369, 0.2661, 0.21045709999999998)"
860,-0.276100,-0.033100,0.215317,0.184357,6.61,300.880000,300.992857,296.164286,302.8,299.2,...,-0.885714,-1.1,1.1,-14.5,0,1,0.663123,-7.485107e-01,18.0,"(-0.2761, -0.0331, 0.2153167, 0.18435710000000..."
859,-0.276100,0.070700,0.156614,0.113414,15.91,300.935714,301.028571,297.365714,303.4,299.3,...,-1.628571,-0.6,1.1,-19.2,0,1,0.568065,-8.229839e-01,43.0,"(-0.2761, 0.0707, 0.15661429999999998, 0.11341..."
1701,-0.274900,-0.083200,0.123571,0.129857,0.00,298.007143,297.985714,292.921429,299.8,295.8,...,-0.742857,-0.5,0.0,4.2,0,1,0.935016,3.546049e-01,15.0,"(-0.2749, -0.0832, 0.1235714, 0.1298571)"
911,-0.251700,-0.048600,0.205171,0.172883,15.25,300.604286,300.685714,295.838571,302.6,298.2,...,1.085714,0.0,-2.2,-33.9,0,1,0.663123,-7.485107e-01,60.0,"(-0.2517, -0.0486, 0.2051714, 0.1728833)"


In [106]:
test.sort_values(by='ndvi_ne')

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,cheat
185,sj,2011,46,2011-11-19,-0.463400,-0.033100,0.102786,0.061100,44.02,300.134286,...,79.384286,44.02,17.635714,2.328571,27.100000,6.285714,31.7,23.3,31.8,"(-0.4634, -0.0331, 0.1027857, 0.0611)"
241,sj,2012,50,2012-12-16,-0.364000,-0.142900,0.067257,0.065857,34.66,299.150000,...,75.081429,34.66,15.682857,2.200000,27.085714,6.300000,32.2,22.8,9.8,"(-0.364, -0.1429, 0.06725714, 0.06585714)"
243,sj,2013,1,2013-01-01,-0.309800,-0.112050,0.143671,0.140814,31.83,298.295714,...,80.552857,31.83,15.980000,2.371429,26.942857,6.585714,31.1,21.7,22.7,"(-0.3098, -0.11205, 0.1436714, 0.1408142999999..."
245,sj,2013,3,2013-01-15,-0.274900,-0.083200,0.123571,0.129857,0.00,298.007143,...,73.492857,0.00,14.311429,1.771429,25.600000,5.500000,28.9,21.1,17.2,"(-0.2749, -0.0832, 0.1235714, 0.1298571)"
104,sj,2010,17,2010-04-30,-0.215400,-0.040850,0.068300,0.056157,43.33,299.662857,...,83.180000,43.33,18.075714,2.542857,27.800000,5.257143,32.8,23.3,21.9,"(-0.2154, -0.04085, 0.0683, 0.05615714)"
134,sj,2010,47,2010-11-26,-0.212000,0.014800,0.281857,0.214729,1.08,299.258571,...,70.932857,1.08,14.990000,2.171429,26.157143,5.700000,29.4,22.2,27.5,"(-0.212, 0.0148, 0.28185709999999997, 0.214728..."
131,sj,2010,44,2010-11-05,-0.205200,0.130150,0.138714,0.134414,10.37,300.325714,...,77.401429,10.37,17.461429,2.642857,27.885714,6.342857,32.2,22.8,61.7,"(-0.2052, 0.13015, 0.13871429999999998, 0.1344..."
256,sj,2013,14,2013-04-02,-0.203250,-0.077833,0.204171,0.178914,6.55,298.035714,...,81.650000,6.55,15.881429,2.828571,26.642857,6.642857,33.3,22.8,17.6,"(-0.20325, -0.07783333, 0.2041714, 0.178914299..."
137,sj,2010,50,2010-12-17,-0.185167,-0.132375,0.209000,0.201657,32.90,298.822857,...,78.514286,32.90,16.184286,3.571429,26.128571,5.942857,31.1,22.8,45.2,"(-0.18516670000000002, -0.132375, 0.209, 0.201..."
133,sj,2010,46,2010-11-19,-0.182650,0.114467,0.109250,0.080243,42.65,298.954286,...,78.030000,42.65,16.238571,3.128571,26.285714,5.000000,30.0,22.8,48.6,"(-0.18265, 0.11446669999999999, 0.10925, 0.080..."


In [118]:
test['cheat'] = test[['ndvi_se', 'ndvi_sw']].apply(tuple,axis=1)

In [119]:
X_test['cheat'] = X_test[['ndvi_se', 'ndvi_sw']].apply(tuple,axis=1)

In [95]:
test_with_index = test.copy()
test_with_index['ind'] = test_with_index.index.values

In [120]:
test.shape

(416, 25)

In [121]:
X_test.shape

(1872, 47)

In [124]:
X_test_to_deliver_because_we_can = X_test.merge(test, how='inner', suffixes=('_original','_pred'), on='cheat')


In [125]:
X_test_to_deliver_because_we_can.shape

(416, 71)

In [73]:
X_test_to_deliver_because_we_can = X_test_to_deliver_because_we_can.drop_duplicates(subset='cheat')

In [127]:
X_test_to_deliver_because_we_can[['city', 'year', 'weekofyear', 'total_cases']].to_csv('luis_vasquinho_ricardo.csv', index=False, encoding='utf8')

In [100]:
X_test_to_deliver_because_we_can.shape

(368, 71)

In [101]:
X_test_to_deliver_because_we_can

,ndvi_ne_original,ndvi_nw_original,ndvi_se_original,ndvi_sw_original,precipitation_amt_mm_original,reanalysis_air_temp_k_original,reanalysis_avg_temp_k_original,reanalysis_dew_point_temp_k_original,reanalysis_max_air_temp_k_original,reanalysis_min_air_temp_k_original,...,reanalysis_precip_amt_kg_per_m2_pred,reanalysis_relative_humidity_percent_pred,reanalysis_sat_precip_amt_mm_pred,reanalysis_specific_humidity_g_per_kg_pred,reanalysis_tdtr_k_pred,station_avg_temp_c_pred,station_diur_temp_rng_c_pred,station_max_temp_c_pred,station_min_temp_c_pred,station_precip_mm_pred
0,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,299.214286,296.118571,307.9,293.7,...,34.11,92.581429,82.29,17.654286,9.428571,27.440000,10.760000,33.8,21.5,11.2
1,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,299.464286,294.831429,307.7,292.2,...,9.10,83.885714,25.30,16.320000,10.157143,27.025000,9.625000,33.0,21.2,8.9
2,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,298.964286,296.417143,306.4,292.5,...,61.09,92.057143,62.14,18.030000,9.557143,26.950000,10.350000,33.4,21.6,22.6
3,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,296.642857,293.638571,306.4,289.3,...,19.60,88.970000,47.80,15.394286,7.828571,26.900000,9.700000,33.3,14.2,4.8
4,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,300.000000,293.892857,308.1,290.5,...,18.93,78.610000,56.30,15.468571,11.771429,27.050000,11.850000,33.5,16.9,3.0
5,0.342020,0.304857,0.332357,0.345729,16.95,298.611429,300.464286,295.065714,309.7,293.4,...,32.92,83.274286,16.95,16.624286,11.157143,27.700000,12.600000,34.0,20.0,92.2
6,0.332217,0.384817,0.356600,0.353917,9.88,298.305714,299.700000,292.085714,309.3,292.2,...,10.50,72.471429,9.88,13.990000,11.828571,26.433333,12.033333,34.2,18.5,0.0
7,0.221814,0.224100,0.273086,0.231243,15.76,298.724286,300.892857,294.812857,309.2,293.2,...,20.30,81.557143,15.76,16.375714,12.642857,27.275000,11.375000,34.0,21.0,20.3
8,0.319043,0.315871,0.326700,0.344057,7.82,299.914286,301.785714,292.691429,311.1,292.3,...,14.21,68.615714,7.82,14.484286,14.485714,27.400000,13.100000,34.7,20.0,NaN
9,0.342014,0.301929,0.314700,0.381000,59.06,300.224286,301.535714,295.200000,310.6,294.1,...,41.10,77.202857,59.06,16.801429,12.671429,28.175000,13.200000,36.2,21.0,8.5


In [229]:
X_test_to_deliver_because_we_can[['city', 'year', 'weekofyear', 'total_cases']].to_csv('luis_vasquinho_ricardo.csv', index=False, encoding='utf8')

In [197]:
X_test_to_deliver_because_we_can = X_test.join(test[['city', 'year', 'weekofyear']], how='right', rsuffix='_pred')